# WebScraping Code INSEE 



In [ ]:
#Import des librairies 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# Data Import 

Import with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Import data from Google drive

In [ ]:
#import data from googledrive
#nomfichier="nomfichier"
#df_search = pd.read_csv('/gdrive/My Drive/{}.csv'.format(nomfichier), encoding='utf-8')

Import manually 

In [ ]:
#other option : put value manually into a dataframe
ar = np.array([[2290,'SAINT-CHRISTOPHE-À-BERRY'], [14240,'PARFOURU L ECLIN'],[79100,'TAIZE']])
df_search = pd.DataFrame(ar,columns=['Code Postal','Nom de la commune'])


In [ ]:
df_search.head()

,Code Postal,Nom de la commune
0,2290,SAINT-CHRISTOPHE-À-BERRY
1,14240,PARFOURU L ECLIN
2,79100,TAIZE


Key_search Creation

In [ ]:
df_search["key_search"]=df_search['Code Postal'] + " " + df_search['Nom de la commune']

In [ ]:
df_search.head()

,Code Postal,Nom de la commune,key_search
0,2290,SAINT-CHRISTOPHE-À-BERRY,2290 SAINT-CHRISTOPHE-À-BERRY
1,14240,PARFOURU L ECLIN,14240 PARFOURU L ECLIN
2,79100,TAIZE,79100 TAIZE


# **Data extraction  from first page of google search:**

**1 - Key_search Creation**

with key_search = Code postal + Nom de la commune

**2 - Launch of google_search()**

- Strengthening's key_search   

    *For each key we addeed " CODE INSEE " for optimize the results.*
- Utilization of **request_and_soup()** 
  
  Take first page of google search
- Utilization of **find_all_for_searched_words()**

  Looking inside the text if the word INSEE it is. 
  NB: We are looking for different format : (INSEE,insee, Insee)

- Utilization of  **find_code_insee()**

  Looking for the code insee in the text.
   
- Output of this function is a list (key_search,codes found)

**3 - Dataframe with all the results (key_search,code found)**

- First filter on dates with **isnotyear()**

- Second Filter on the "codes postaux" with **extractcodepostalfromkey()** and **dropcodepostal()**.

- Find the correct code_insee :
  We are taking the most result found with **most_codeinsee()**

**4 - Export file on google drive**


In [ ]:
#key_search_list
search_list = []
for i in df_search['key_search']:
  search_list.append(i)

In [ ]:
#Take text from searching page of google
def request_and_soup(key):
    word = key + " CODE INSEE"
    print('\n\n\n')
    print(word)
    base_url = "https://www.google.com/search"
    source = requests.get(base_url+'?q='+word+" "+"&oq="+word+"aqs=chrome.0.69i59j46j69i59j0l2j69i60l3.1087j0j7&sourceid=chrome&ie=UTF-8") 
    html_doc = source.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup

In [ ]:
#Looking for the word "Insee" inside the text 
def find_all_for_searched_words(soup,searched_word):
  results = []
  for s in searched_word:
    results += soup.body.find_all(string=re.compile('{0}'.format(searched_word)), recursive=True)
  return results

In [ ]:
#Looking for the code INSEE
def find_code_insee(results):
  code_insee =[]
  for content in results: 
    #lowercase for all content
    content = content.lower()
    #split element with insee word
    part_de_phrases = content.split('insee')
    #looking for numeric part
    code_insee += re.findall('([0-9]{4,5}|[0-9]{1}[a-zA-Z0-9]{1}[0-9]{3})',str(part_de_phrases))
  return code_insee

La fonction google_search  (Main fonction)

In [ ]:
def google_search(word_list):
  result_brut = []
  for key in word_list:

    soup = request_and_soup(key)

    searched_word=["INSEE","Insee","insee"]

    for s in searched_word :
      results = find_all_for_searched_words(soup,s)

      code_insee = find_code_insee(results)
    
      result_brut.append((key,code_insee))
        

  return result_brut

In [ ]:
resultat_list = google_search(search_list)

Result sent into a dataframe

In [ ]:
df_results = pd.DataFrame(resultat_list,columns=['key_search','code_insee'])
df_results = df_results.groupby('key_search').last()
df_results

Fonction **isnotyear()** :

Filter of the year presented into the result.

In [ ]:
def isnotyear(code):
  if len(code) == 4 and (int(code) > 1900 and int(code) < 2050):
    return False
  else:
    return True

In [ ]:
def drop_date(list_code):
  return list(filter(isnotyear, list_code))


In [ ]:
df_results['code_insee'] = df_results['code_insee'].apply(drop_date)

Filter on the "Code Postaux"

In [ ]:
def dropcodepostal(vraicodepostal,listcodeinsee):
  nouvellelistecodeinsee=[]
  for i in listcodeinsee:
    if i != vraicodepostal  and i != str(str('0') + vraicodepostal):
      nouvellelistecodeinsee.append(i)
  return nouvellelistecodeinsee

In [ ]:
def extractcodepostalfromkey(key):
  codepostal=key[:5]
  return codepostal.strip()


In [ ]:
df_results=df_results.reset_index()

In [ ]:
row_iterator = df_results.iterrows()

for i in row_iterator:
  codepostal = extractcodepostalfromkey(i[1][0])
  print(codepostal)
  newlistcodeinsee=dropcodepostal(codepostal,i[1][1])
  print(newlistcodeinsee)
  df_results.loc[i[0], 'code_insee'] = newlistcodeinsee


In [ ]:
df_results

 **most_code_insee()**, take the most code found

In [ ]:
from collections import Counter
def most_code_insee(list_code):
  if list_code != []:
    return Counter(list_code).most_common(1)[0][0]
  else:
    return np.nan

In [ ]:
df_results['code_insee']=df_results['code_insee'].apply(most_code_insee)


Export into Google drive with a CSV file


In [ ]:
df_results.to_csv('webscraping_results3.csv',header=True)

In [ ]:
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!cp webscraping_results.csv "/gdrive/My Drive/"